# Laboratorio de regresión - 4

//                |   |
:----------------|---|
| **Nombre**     |  Isabela Torres-Septien Uribe |
| **Fecha**      |  09/09/2025
| **Expediente** |   730667 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

## Regresion lineal 1, sin penalización

In [5]:
df = pd.read_csv("Advertising.csv")
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [26]:
X = df[["TV", "radio", "newspaper"]]
y = df["sales"]

In [ ]:


lr = LinearRegression()
lr.fit(X, y)

b0 = lr.intercept_
b1, b2, b3 = lr.coef_

print(f"sales = {b0:.4f} + {b1:.4f}*TV + {b2:.4f}*radio + {b3:.4f}*newspaper")


sales = 2.9389 + 0.0458*TV + 0.1885*radio + -0.0010*newspaper


In [29]:
x_bar = np.mean(X)
y_bar = np.mean(y)
n = len(X)

y_hat = lr.predict(X)
y_hat

array([20.52397441, 12.33785482, 12.30767078, 17.59782951, 13.18867186,
       12.47834763, 11.72975995, 12.12295317,  3.72734086, 12.55084872,
        7.0322992 , 17.28512918, 10.57712073,  8.82630048, 18.43436638,
       20.81929952, 12.82365674, 23.22495716,  9.95168206, 14.16607293,
       18.10076728, 14.7405382 ,  6.4891503 , 16.5459329 ,  8.14651887,
       15.6100386 , 14.98951429, 17.05167344, 19.41053803,  9.14402389,
       21.6339338 , 11.3460929 ,  7.63888314, 18.86426829,  7.57483051,
       17.00682618, 23.40590052, 15.62347779,  9.90868103, 20.44761039,
       16.37766467, 17.2959832 , 21.59580326, 13.96385684,  8.88787996,
       15.16152314,  8.87338673, 21.7226299 , 16.26362018,  8.1681656 ,
       12.63121132,  9.33981296, 20.66297563, 19.94469957, 20.37443008,
       21.2926106 ,  8.52771254, 12.77458802, 21.89805198, 18.13348698,
        5.74215558, 22.89067208, 16.78426073, 13.21069202, 16.97773556,
        7.84904532,  9.01603163, 12.0370073 , 18.97657924, 21.10

In [32]:
#Evaluación del modelo
r_score = r2_score(y, y_hat)
MSE = mean_squared_error(y, y_hat)

print("R²:", r_score)
print("MSE:", MSE)


R²: 0.8972106381789522
MSE: 2.784126314510936


## Regresion con penalización de ridge

Los coeficientes son similares a los obtenidos con regresión lineal estándar, por eso los implementaremos en la regresión de Lasso.

In [44]:
#ya habíamos definido X, y
from sklearn.pipeline import Pipeline

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.4, random_state=137)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


ridge = Ridge(alpha=1.0, fit_intercept=True)
ridge.fit(X_scaled, y)

coef = ridge.coef_ / scaler.scale_
r_intercept = ridge.intercept_ - np.dot(scaler.mean_, coef)

# Imprimimos ecuación
print(f"ridge sales = {r_intercept:.4f} + {coef[0]:.4f}*TV + {coef[1]:.4f}*radio + {coef[2]:.4f}*newspaper")



ridge sales = 2.9862 + 0.0455*TV + 0.1875*radio + -0.0007*newspaper


In [47]:
y_hat_ridge = ridge.predict(X_scaled)
r2_ridge = r2_score(y, y_hat_ridge)
mse_ridge = mean_squared_error(y, y_hat_ridge)
print("R^2", r2_ridge)
print("MSE", mse_ridge)

R^2 0.8971891437493418
MSE 2.7847085071234132


## Regresion con penalización de Lasso

In [46]:
#Ya habiamos definido las variables X, y, sepamos en train y test y escalamos así que ahora utilizamos Lasso

alpha = 0.1 
lasso = Lasso(alpha=alpha, fit_intercept=True)
lasso.fit(X_scaled, y)

b0 = lasso.intercept_ - np.dot(scaler.mean_, lasso.coef_ / scaler.scale_)
b1, b2, b3 = lasso.coef_ / scaler.scale_

print(f"lasso sales = {b0:.4f} + {b1:.4f}*TV + {b2:.4f}*radio + {b3:.4f}*newspaper")


lasso sales = 3.2328 + 0.0446*TV + 0.1816*radio + 0.0000*newspaper


In [48]:
y_hat_lasso = lasso.predict(X_scaled)
r2_lasso = r2_score(y, y_hat_lasso)
mse_lasso = mean_squared_error(y, y_hat_lasso)
print("R^2", r2_lasso)
print("MSE", mse_lasso)

R^2 0.896494232900825
MSE 2.8035306842954344


## Conclusiones

Al comparar los coeficientes unos con otros, este es el modelo donde se ve más claro que newspaper se va y no entra dentro de las regresiones. por lo tanto este calor seería el que eliminaríamos de nuestros datos. En la regresión linal sin penalizasiones se ve que es el menor y en la regresión de ridge, aunque es más exacta es hasta la regresión de ridge donde realmente el valor de newspapaer es basicamente nulo. Junto a esto analizamos R^2 y el MSE. 

Analizando R^2
- En la primera regresion lineal podemos ver como los tres modelos son muy cercanos a 1, sin embrago el modelo más cercano es el de ridge el cual, tiene menos variabilidad al ser el valor más alto entre los tres modelos. Por otro lado, el modelos de lasso es el que tiene menor ajuste tiene, aunque la diferencia entre un modelo y otro es minimo.

Analizando MSE
- En el modelo de MSE estamos calculando la magnitud del error, en general la magnitud del error es baja en los tres modelos. Sin embargo, es más baja en la primera regresión lineal, esto probablemente se debe a que el modelo no tiene penalizaciones. A pesar de esto, todos los los modelos muestran un MSE similar, siendo el lasso más alto

FInalmente, al comparar los tres modelos podemos definir cual es el más optimo, aunque cada uno se usa en diferentes aspectos de la regresión lineal, personalmete el modelo lasso, se me hizo más facil aplicar, sin embargo, al comparar los resultados creo que para tener modelos con mejor variabilidad y poder calcular el MSE utilizaría el modelo ridge